### TODO:
- Criar função check_double para checar se após uma alocação, alguém está com turno dobrado.
    - Caso sim, alocar folga novamente
- Repetir lógica do solve até todos os shifts estarem alocados ou encontrar um erro
- Criar lógica para lidar com soft constraints
- Implementar backtracking
- Refactor code
    - Strings vs Custom Classes in different methods
    - Once functional, move classes and usage to separate scripts
- Create tests
- Create CI pipeline
- days_off_per_week:
    - Actually consider no_days
    - Handle substitutes

In [1]:
from enum import Enum
from typing import List
from dateutil.relativedelta import relativedelta
from datetime import datetime, date
import pandas as pd
import numpy as np
from copy import deepcopy, copy
import bisect # Used for fast searching of first x greater than K in a list

In [2]:
#Criar dataframe contendo posições e nomes
morning_workers = ['CHEF1', 'COZ1', 'COZ2', 'ASG1', 'ASG2', 'AUX1', 'CONF1', 'PIZ1', 'GARD1', 'MASSA1']
evening_workers = ['CHEF2', 'COZ3', 'COZ4', 'ASG3', 'ASG4', 'AUX2', 'AUXPIZ1', 'PIZ2', 'GARD2', 'MASSA2']
workers = morning_workers + evening_workers
positions = [x[:-1] for x in workers]
df = pd.DataFrame({'position': positions, 'worker': workers})
#Update caso particular
df.loc[df['worker'] == 'AUXPIZ1', 'position'] = 'PIZ'
#df = pd.concat([df, pd.DataFrame([['CONF','AUXPIZ1']], columns=df.columns)], axis = 0)
#df = pd.concat([df, pd.DataFrame([['AUXPIZ','CONF1']], columns=df.columns)], axis = 0)

In [3]:
# These definitions will help set constraints in an intuitive manner by the user
MONDAY = 0
TUESDAY = 1
WEDNESDAY = 2
THURSDAY = 3
FRIDAY = 4
SATURDAY = 5
SUNDAY = 6

In [4]:
class CalendarDay():
    """
    A CalendarDay represents a day of the month.
    Attrs:
        - Date
        - Day of the week
        - Number of Shifts
        - Week of the year
    """
   
    
    def __init__(self, date: date):
        self.date = date
        self.date_alias = self.date.strftime('%a%d')
        self.week_day = self.date.weekday()
        self.week = self.date.strftime('%W')
        self.next_day = None
    
    def __repr__(self):
        return f"{self.date_alias}"
        

class Shift():
    """
    A Shift needs a set number of workers per position (slots) to be filled.
    """
    def __init__(self, cal_day: CalendarDay, no: int, positions: List[str], no_workers_per_position: dict, candidates):
        self.cal_day = cal_day
        self.shift_name = str(self.cal_day.date_alias) + ' S' +str(no).zfill(2)
        self.week_day = cal_day.week_day
        self.week = cal_day.week
        self.no = no
        self.workers = []
        self.slots_to_fill = no_workers_per_position
        self.filled_slots = {}
        self.candidates = candidates
        for position in positions:
            #self.slots_to_fill[str(position)] = no_workers_per_position
            self.filled_slots[position] = set()
        
    def update(self, slots_to_fill, candidates):
        """
        Allows the user to specify slots_to_fill using a dictionary with keys: positions 
        and values: number of workers for that position; and the candidates dictionary with
        keys: position, values: set of worker names 
        """
        self.slots_to_fill = slots_to_fill
        self.candidates = candidates
        for position in self.slots_to_fill.keys():
            self.filled_slots[position] = set()
    
    def __repr__(self):
        return self.shift_name
    

class Worker():
    def __init__(self, name: str, position: str, availability: List[Shift] = []):
        self.name = name
        self.position = position
        self.availability = []
        self.no_shifts = 0
        self.shift_preference = 100
        self.days_off = []
        
    def __repr__(self):
        return self.name
    
    def __str__(self):
        return self.name



class Schedule():
    """
    Class which contains the calendar and the solver (AC3).
    Initializes the days and shifts. Sets requirements for each shift in terms of workers and positions. Executes solver.
    Users should pass the positions names and requirements, as well as the worker list.
    attrs:
        - start_date: 'YYYYMMDD'
        - end_date: 'YYYYMMDD', inclusive
        - number_of_shifts: number of shifts within a day
        - number_of_positions: number of distinct roles that should be allocated within each shift
        - calendar_days
        
    methods:
        - add_contraint
        - remove_constraint
        - solve
    """
    
    next_month = (date.today() + relativedelta(months=1))
    first_day = next_month + relativedelta(days=-next_month.day + 1)
    last_day = first_day + relativedelta(months=1, days=-1)
    
    def __init__(self, 
                 no_workers_per_position: dict,
                 start_date: str = first_day,
                 end_date: str = last_day, 
                 no_shifts_per_day: int = 2, 
                 df: pd.DataFrame = pd.DataFrame({'position': [], 'worker': []}),
                 
                ):
        self.start_date = start_date
        self.end_date = end_date
        self.constraints = {}
        self.df = df
        self.no_workers_per_position = no_workers_per_position
        self.positions, self.workers = self.get_workers_and_positions(df)
        self.no_positions = len(positions)
        self.calendar_days = [CalendarDay(x) for x in pd.date_range(start_date, end_date, freq='1d')]
        for i in range(0, len(self.calendar_days)):
            if i < len(self.calendar_days) - 1:
                self.calendar_days[i].next_day = self.calendar_days[i + 1]
        #candidates_per_position should be combined with worker availability
        self.candidates_per_position = dict(df.groupby('position')['worker'].unique().apply(set))
        #Initialize shifts
        self.shifts = [] 
        self.worker_allocation = {}
        print(f"positions: {self.positions}")
        for cal_day in self.calendar_days:
            for shift_no in range(1, no_shifts_per_day + 1):
                self.shifts.append(Shift(cal_day,shift_no, self.positions, self.no_workers_per_position, self.candidates_per_position))
        #Initialize worker availability
        for worker in self.workers:
            worker.availability = copy(self.shifts)
            self.worker_allocation[worker.name] = 0
        #Set substitutes
        self.substitutes = {'CONF': 'AUXPIZ1', 'AUXPIZ': 'CONF1'}
        
    
    def get_workers_and_positions(self, df):
        """
        Takes a pandas DataFrame containing name of the worker and position.
        Assumes there are not duplicate worker names.
        Returns all the positions and workers as lists.
        """
        positions = list(df['position'].unique())
        workers = []
        for row in df.itertuples():
            worker = Worker(row[2],row[1])
            workers.append(worker)
        return positions, workers
    
    
    
    def add_schedule_constraint(self, kind='days_off_per_week', **kwargs):
        """
            - kind: 
                worker_to_shiftno 
                    - soft constraint - try to enforce after hard constraints
                    - worker can only work on shift n 
                    - kwargs: worker_name, shift_no
                days_off_per_week 
                    - hard constraint
                    - kwargs: no_days
                days_off_per_month 
                    - hard constraint
                    - kwargs: no_days
                days_off_per_weekday_per_month 
                    - hard constraint
                    - kwargs: no_days, weekday                    
                consecutive_days_off 
                    - soft constraint
                    - kwargs: no_days
                day_off_after_doubling 
                    - hard_constraint
                    - kwargs: flag
        """
        if kind == 'days_off_per_week':
            try:
                no_days = kwargs['no_days']
                self.constraints[kind] = {'no_days': no_days}
            except KeyError:
                print(f'You must specify no_days when kind is {kind}!')
        elif kind == 'days_off_per_month':
            try:
                no_days = kwargs['no_days']
                self.constraints[kind] = {'no_days': no_days}
            except KeyError:
                print(f'You must specify no_days when kind is {kind}!')
        elif kind == 'days_off_per_weekday_per_month':
            try:
                no_days = kwargs['no_days']
                week_day = kwargs['week_day']
                self.constraints[kind] = {'no_days': no_days, 'week_day': week_day}
            except KeyError:
                print(f'You must specify no_days and week_day when kind is {kind}!')
        elif kind == 'day_off_after_doubling':
            self.constraints[kind] = True
        elif kind == 'consecutive_days_off':
            try:
                no_days = kwargs['no_days']
                self.constraints[kind] = {'no_days': no_days}
            except KeyError:
                print(f'You must specify no_days when kind is {kind}!')
        #Implement consecutive_days_off and worker_to_shiftno
    
    def update_number_of_shifts_for_particular_day(self):
        pass
        
    def remove_worker_availability_by_day(self, worker: Worker, day: CalendarDay):
        shifts = self.get_shifts_by_calendar_day(day)
        #print(shifts)
        worker.days_off.append(day.date)
        for shift in shifts:
            worker.availability.remove(shift)
    
    # -----------------  Day off functions ----------------------- #
    def compute_interval_between_days_off(self):
        results = []
        for worker in self.workers:
            days_off = sorted(worker.days_off)
            cp_days_off = copy(days_off)
            intervals = []
            while len(cp_days_off) > 1: #If worker has been allocated more than 1 day off
                #Compute time between days off
                interval = cp_days_off[-1] - cp_days_off[-2]
                intervals.append(interval)
                cp_days_off.pop()
            results.append([worker, intervals, days_off])
        return results
    
    def set_new_day_off_within_x_days(self, worker, start_date, x, debug=False):
        cal_day = self.get_calday_by_date(start_date)
        flag_able_to_allocate = False
        # Iterate for x next days and verify if there is a day when we can fulfill the slot requirements without that worker
        for i in range(x):
            cal_day = cal_day.next_day
            available_workers = self.get_workers_by_calday_availability(cal_day)
            position_workers = set([x.name for x in self.get_workers_by_position(worker.position)])
            intersection = available_workers.intersection(position_workers)
            shift = self.get_shifts_by_calendar_day(cal_day)[0]
            required_slots = shift.slots_to_fill[worker.position]
            if worker.name in intersection:
                intersection.remove(worker.name)
            if len(intersection) >= required_slots:
                #Remove availability
                flag_able_to_allocate = True
                self.remove_worker_availability_by_day(worker, cal_day)
                #Save day
                break
            if debug:
                print(f"Available Workers: {available_workers}")
                print(f"Position Workers: {position_workers}")
                print(f"Intersection: {intersection}")
                print(f"Required slots: {required_slots}")
        if flag_able_to_allocate:
            print(f'Success. Worker {worker} has been given a new day off at {cal_day}.')
            return cal_day
        else:
            print(f'Unable to allocate day off for worker {worker} in the specified range.'.upper())
            return None
        
     # ------------------------------------------------------------------- #
        
     # ----------------- Helper querying functions ----------------------- #
    def get_shifts_by_number(self, no):
        return [x for x in self.shifts if x.no == no]
    
    def get_shifts_by_calendar_day(self, cal_day):
        return [x for x in self.shifts if x.cal_day == cal_day]
    
    def get_shifts_by_day_of_the_week(self, week_day):
        return [x for x in self.shifts if x.week_day == week_day]
    
    def get_shift_by_name(self, name):
        return [x for x in self.shifts if x.shift_name == name]
    
    def get_calday_by_day_of_the_week(self, week_day):
        return [x for x in self.calendar_days if x.week_day == week_day]
    
    def get_calday_by_date(self, date):
        return [x for x in self.calendar_days if x.date == date][0]
    
    def get_workers_by_calday_availability(self, calday):
        """Returns worker names"""
        shifts = self.get_shifts_by_calendar_day(calday)
        workers = set()
        for shift in shifts:
            for worker in self.workers:
                if shift in worker.availability:
                    workers.add(worker.name)
        return workers
    
    def get_workers_by_position(self, position):
        "Returns a list of Workers for that position"
        return [x for x in self.workers if x.position == position]
    
    def get_worker_by_name(self, name):
        return [x for x in self.workers if x.name == name][0]
        
    # ------------------------------------------------------------------- #
    
    def allocate(self, days: List[CalendarDay]):
        """Allocates available workers in the specified days"""
        for day in days:
            print(f"Day: {day}")
            shifts = self.get_shifts_by_calendar_day(day)
            available_workers = self.get_workers_by_calday_availability(day)
            print(f'Available workers: {available_workers}')
            for shift in shifts:
                for position in shift.slots_to_fill.keys():
                    n_workers = shift.slots_to_fill[position]
                    position_candidates = set(shift.candidates[position])
                    new_candidates = list(available_workers.intersection(position_candidates))
                    new_candidates = sorted(new_candidates, key=lambda x: self.worker_allocation.get(x), reverse=True)
                    while len(shift.filled_slots[position]) < n_workers:
                        if len(new_candidates) > 0:
                            min_allocation = self.worker_allocation.get(new_candidates[-1])
                            ideal_worker_to_allocate = []
                            alternative_worker_to_allocate = []
                            #TODO
                            #Here, I should first verify if I can allocate the worker with min_allocation
                            #if not, try the next min_allocation
                            for worker in new_candidates:
                                if self.worker_allocation.get(worker) == min_allocation:
                                    worker_instance = self.get_worker_by_name(worker)
                                    if worker_instance.shift_preference == shift.no:
                                        ideal_worker_to_allocate.append(worker)
                                    else:
                                        alternative_worker_to_allocate.append(worker)
                            if len(ideal_worker_to_allocate) > 0:
                                #Set worker_to_allocate as the worker with minimum allocation 
                                #that usually works that shift
                                worker_to_allocate = ideal_worker_to_allocate[-1]
                            else:
                                #Set worker_to_allocate as the worker with minimum allocation 
                                #that doesn't usually work that shift
                                worker_to_allocate = alternative_worker_to_allocate[-1]
                            shift.filled_slots[position].add(worker_to_allocate)
                            self.worker_allocation[worker_to_allocate] += 1
                            new_candidates.remove(worker_to_allocate)
                        else:
                            shift.filled_slots[position].add(self.substitutes[position])
    
    def solve(self, debug=False):
        #Start by looking at days_off_per_weekday_per_month
        if 'days_off_per_weekday_per_month' in self.constraints.keys():
            no_days = self.constraints['days_off_per_weekday_per_month']['no_days']
            week_day = self.constraints['days_off_per_weekday_per_month']['week_day']
            #Get weekdays
            days = self.get_calday_by_day_of_the_week(week_day)
            if debug:
                print('--------------------------------------------')
                print('Days off per weekday')
                print('Days: ', days)
            #Crete auxiliary varibles
            calendario_folga = {}
            folgados = set()
            #Each person has to have at least 1 day off on a sunday per month
            for day in days:
                cant_have_a_day_off = set()
                if debug:
                    print(f'\nDay:{day}')
                for position in self.positions:
                    if day not in calendario_folga.keys():
                        calendario_folga[day] = {}
                    candidatos_a_folgado = list(set(self.get_workers_by_position(position)) - folgados - cant_have_a_day_off)
                    if debug:
                        print(f"day:{day}, position:{position}, candidatos_a_folgado: {candidatos_a_folgado}, cant_have_a_day_off: {cant_have_a_day_off}")
                    if len(candidatos_a_folgado) > 0:
                        folgado = np.random.choice(candidatos_a_folgado)
                        if debug:
                            print(f'Folgado escolhido: {folgado}')
                            print(f'Disponibilidade do folgado escolhido: {folgado.availability}')
                        calendario_folga[day][position] = folgado
                        try:
                            self.remove_worker_availability_by_day(folgado, day)
                            # The substitute can't have a day off in the same day as the position it needs to fill
                            if self.substitutes.get(position):
                                cant_have_a_day_off.add(self.substitutes.get(position))
                            folgados.add(folgado)
                            if debug:
                                print(f'Removed availability for worker {folgado} in day {day}')
                        except Exception as e:
                            print(f"Exception {e}")
                            print(folgado, day, folgado.availability)
                            print('\n')
                        
            print('Calendario Folga: ')
            print(calendario_folga)
            #Make sure everyone has a day off on a sunday
            temp = []
            for v in calendario_folga.values():
                for v1 in v.values():
                    temp.append(v1)
            assert(len(set(self.workers) - set(temp)) == 0)
            #First allocation
            print('\n----------------------------------------\n')
            print('First allocation')
            self.allocate(days)
        print('\n----------------------------------------\n')
        
        #Maybe do this after the second day off of the week
        if 'day_off_after_doubling' in self.constraints.keys():
            worker_names = [x.name for x in self.workers]
            #Check days in which employees had two shifts
            doubled = []
            for day in self.calendar_days:
                shifts = self.get_shifts_by_calendar_day(day)
                for worker_name in worker_names:
                    counter = 0
                    for shift in shifts:
                        vals = flat_list = [elem for s in shift.filled_slots.values() for elem in s]
                        if worker_name in vals:
                            counter += 1
                    if counter == 2:
                        doubled.append([day, worker_name])
            print(f"Doubled: {doubled}")
            #Assign day off the next day
            print("Assigning day off after doubling...")
            days = []
            for day, worker_name in doubled:
                next_day = day.next_day
                worker = self.get_worker_by_name(worker_name)
                self.remove_worker_availability_by_day(worker, next_day)
                days.append(next_day)
            #Allocate day off after doubling
            self.allocate(days)
            
        if 'days_off_per_week' in self.constraints.keys():
            no_days = self.constraints['days_off_per_week']['no_days'] #TODO: Where is this used? Currently, just checking if the last day off occurred more than 6 days ago and allocating one more if so
            results = self.compute_interval_between_days_off()
            days = []
            for worker, intervals, days_off in results:
                #If only one day off has been allocated thus far
                if len(intervals) == 0: #Only one day off so far, therefore no interval to compute
                    #Get next available day and turn it into a day_off if there are enough substitutes
                    day = self.set_new_day_off_within_x_days(worker, days_off[0], 6) # x is set as 6 because the constraint specifies the number of days off per week, so we look 6 days ahead.
                    if day is not None:
                        days.append(day)
                #Multiple days off
                else:
                    #Unpack every interval found for each worker and evaluate if it is necessary to introduce a day_off somewhere
                    for idx, val in enumerate(intervals):
                        no_days_between_days_off = val.days
                        if no_days_between_days_off > 6:
                            day = self.set_new_day_off_within_x_days(worker, days_off[idx], 6)
                            if day is not None:
                                days.append(day)
            self.allocate(days)
            
                
    
    def __repr__(self):
        output_str = 'Schedule for '
        output_str += f'{self.start_date} '
        output_str += f'to {self.end_date}:\n'
        output_str += f'\t- Positions: {" ".join(self.positions)}\n'
        output_str += f'\t- Workers: {" ".join([x.name for x in self.workers])}\n'
        return output_str
    



### Create Generic Schedule

In [5]:
no_workers_per_position = {
    'CHEF': 1,
    'COZ': 2,
    'ASG': 2,
    'AUX': 1,
    'CONF': 1,
    'PIZ': 1,
    'GARD': 1,
    'MASSA': 1
}

In [6]:
#Create generic schedule
schedule = Schedule(df=df, 
                    no_workers_per_position=no_workers_per_position, 
                    start_date=datetime(2023,9,1).date(), 
                    end_date=datetime(2023,9,30).date())
schedule

positions: ['CHEF', 'COZ', 'ASG', 'AUX', 'CONF', 'PIZ', 'GARD', 'MASSA']


Schedule for 2023-09-01 to 2023-09-30:
	- Positions: CHEF COZ ASG AUX CONF PIZ GARD MASSA
	- Workers: CHEF1 COZ1 COZ2 ASG1 ASG2 AUX1 CONF1 PIZ1 GARD1 MASSA1 CHEF2 COZ3 COZ4 ASG3 ASG4 AUX2 AUXPIZ1 PIZ2 GARD2 MASSA2

### Assign Shift preference to workers

In [7]:
for worker in schedule.workers:
    if worker.name in morning_workers:
        worker.shift_preference = 1
    elif worker.name in evening_workers:
        worker.shift_preference = 2

### Add some constraints

In [8]:
#Add some constraints
#Each person has to have 2 days off per week
schedule.add_schedule_constraint(kind='days_off_per_week', no_days=2)
#Each person has to have at least 1 sunday off per month
schedule.add_schedule_constraint(kind='days_off_per_weekday_per_month', no_days=1, week_day=SUNDAY)
#If possible, whoever has the sunday off has monday off as well
schedule.add_schedule_constraint(kind='consecutive_days_off', no_days = 2)
#The same person should only have two shifts on the same day if their next is a day off
schedule.add_schedule_constraint(kind='day_off_after_doubling')
schedule.constraints

{'days_off_per_week': {'no_days': 2},
 'days_off_per_weekday_per_month': {'no_days': 1, 'week_day': 6},
 'consecutive_days_off': {'no_days': 2},
 'day_off_after_doubling': True}

In [9]:
schedule.shifts[5].week

'35'

### Modify specific shifts

##### Update every SUNDAY EVENING

In [10]:
sundays = schedule.get_shifts_by_day_of_the_week(SUNDAY)
sunday_evenings = [x for x in sundays if x.no == 2]
sunday_evenings

[Sun03 S02, Sun10 S02, Sun17 S02, Sun24 S02]

In [11]:
for shift in sunday_evenings:
    shift.update(slots_to_fill = {'CHEF': 1, 'CUSTOM': 2}, 
                 candidates = {'CHEF': set(['CHEF1', 'CHEF2']), 
                               'CUSTOM': set(['PIZ1','PIZ2','AUXPIZ1'])
                              })

##### Inspect shifts

In [12]:
shift = schedule.shifts[0]
print(shift)
print(shift.slots_to_fill)
print('\n')
print(shift.candidates)

Fri01 S01
{'CHEF': 1, 'COZ': 2, 'ASG': 2, 'AUX': 1, 'CONF': 1, 'PIZ': 1, 'GARD': 1, 'MASSA': 1}


{'ASG': {'ASG1', 'ASG3', 'ASG4', 'ASG2'}, 'AUX': {'AUX1', 'AUX2'}, 'CHEF': {'CHEF1', 'CHEF2'}, 'CONF': {'CONF1'}, 'COZ': {'COZ2', 'COZ4', 'COZ1', 'COZ3'}, 'GARD': {'GARD2', 'GARD1'}, 'MASSA': {'MASSA2', 'MASSA1'}, 'PIZ': {'PIZ1', 'AUXPIZ1', 'PIZ2'}}


In [13]:
shift = schedule.shifts[5]

In [14]:
print(shift)
print(shift.slots_to_fill)
print('\n')
print(shift.candidates)

Sun03 S02
{'CHEF': 1, 'CUSTOM': 2}


{'CHEF': {'CHEF1', 'CHEF2'}, 'CUSTOM': {'PIZ1', 'AUXPIZ1', 'PIZ2'}}


### Solver

In [15]:
#Se AUXPIZ folgar com outro PIZ, não dá pra fechar o turno de domingo a noite
schedule.solve(debug=False)

Calendario Folga: 
{Sun03: {'CHEF': CHEF2, 'COZ': COZ2, 'ASG': ASG2, 'AUX': AUX2, 'CONF': CONF1, 'PIZ': PIZ1, 'GARD': GARD1, 'MASSA': MASSA2}, Sun10: {'CHEF': CHEF1, 'COZ': COZ3, 'ASG': ASG4, 'AUX': AUX1, 'PIZ': AUXPIZ1, 'GARD': GARD2, 'MASSA': MASSA1}, Sun17: {'COZ': COZ1, 'ASG': ASG1, 'PIZ': PIZ2}, Sun24: {'COZ': COZ4, 'ASG': ASG3}}

----------------------------------------

First allocation
Day: Sun03
Available workers: {'ASG3', 'PIZ2', 'AUXPIZ1', 'MASSA1', 'ASG1', 'GARD2', 'COZ4', 'COZ1', 'CHEF1', 'ASG4', 'AUX1', 'COZ3'}
Day: Sun10
Available workers: {'ASG3', 'AUX2', 'PIZ2', 'COZ2', 'CONF1', 'ASG2', 'CHEF2', 'ASG1', 'GARD1', 'COZ1', 'COZ4', 'MASSA2', 'PIZ1'}
Day: Sun17
Available workers: {'ASG3', 'AUX2', 'COZ2', 'CONF1', 'ASG2', 'AUXPIZ1', 'MASSA1', 'CHEF2', 'GARD2', 'GARD1', 'COZ4', 'CHEF1', 'ASG4', 'AUX1', 'MASSA2', 'PIZ1', 'COZ3'}
Day: Sun24
Available workers: {'AUX2', 'PIZ2', 'COZ2', 'CONF1', 'ASG2', 'AUXPIZ1', 'MASSA1', 'ASG1', 'CHEF2', 'GARD1', 'ASG4', 'COZ1', 'CHEF1', 'GARD2

In [16]:
schedule.shifts[5].slots_to_fill

{'CHEF': 1, 'CUSTOM': 2}

In [ ]:
schedule.shifts[5]

In [ ]:
schedule.shifts[4].filled_slots

In [ ]:
schedule.shifts[5].filled_slots

In [ ]:
for shift in sundays:
    print('\n--------------------------\n')
    print(f"Shift: {shift}")
    print(f'Slots to fill: {shift.slots_to_fill}')
    print(f'Filled slots: {shift.filled_slots}')

In [ ]:
mondays = schedule.get_shifts_by_day_of_the_week(MONDAY)

In [ ]:
for shift in mondays:
    print('\n--------------------------\n')
    print(f"Shift: {shift}")
    print(f'Slots to fill: {shift.slots_to_fill}')
    print(f'Filled slots: {shift.filled_slots}')

---
### Compute Intervals between days off

In [ ]:
results = []
for worker in schedule.workers:
    #worker_name = worker.name
    days_off = sorted(worker.days_off)
    cp_days_off = copy(days_off)
    intervals = []
    while len(cp_days_off) > 1: 
        #Compute time between days_off
        interval = cp_days_off[-1] - cp_days_off[-2]
        intervals.append(interval)
        cp_days_off.pop()
    results.append([worker, intervals, days_off])
    #day_availability = [x.cal_day.date for x in worker.availability]
    print(f"Worker: {worker.name},\n  Days off: {days_off},\n Intervals: {intervals}")
    print('-------------------------------------------------------------------------')

In [ ]:
def set_new_day_off_within_x_days(worker, start_date, x, debug=True):
    next_x_days = []
    cal_day = schedule.get_calday_by_date(start_date)
    flag_able_to_allocate = False
    for i in range(x):
        cal_day = cal_day.next_day
        next_x_days.append(cal_day)
        available_workers = schedule.get_workers_by_calday_availability(cal_day)
        position_workers = set([x.name for x in schedule.get_workers_by_position(worker.position)])
        intersection = available_workers.intersection(position_workers)
        shift = schedule.get_shifts_by_calendar_day(cal_day)[0]
        required_slots = shift.slots_to_fill[worker.position]
        if worker.name in intersection:
            intersection.remove(worker.name)
        if len(intersection) >= required_slots: #Actually, not 0, number of workers needed to fill a position
            #Remove availability
            flag_able_to_allocate = True
            #print('Good to go')
            break
        if debug:
            print(f"Available Workers: {available_workers}")
            print(f"Position Workers: {position_workers}")
            print(f"Intersection: {intersection}")
            print(f"Required slots: {required_slots}")
    if flag_able_to_allocate:
        print('Good to go')
    else:
        print(f'Unable to allocate day off for worker {worker.name} in the specified range.'.upper())

In [ ]:
x = 6 # Number of days ahead to look for a day off
for item in results:
    worker = item[0]
    intervals = item[1]
    days_off = item[2]
    #If only one day off
    if len(intervals) == 0: #Only one day off so far, therefore no interval to compute
        #Get next available day and turn it into a day_off if there are enough substitutes
        set_new_day_off_within_x_days(worker, days_off[0], x)
        print(item)
        print('---------------\n')
    #Multiple days off
    else:
        #Unpack every interval found for each worker and evaluate if it is necessary to introduce a day_off somewhere
        for idx, val in enumerate(intervals):
            print('\n\nELSE')
            no_days_between_days_off = val.days
            if no_days_between_days_off > x:
                set_new_day_off_within_x_days(worker, days_off[idx], x)
                print('END ELSE')

In [ ]:
schedule.get_calday_by_date(days_off[0])